In [1]:
# ================================================================
# LULC (SVM) for Omo Forest Reserve, Nigeria (2020 & 2024)
# Landsat 8 OLI/TIRS C2 L2 + ESA Dynamic World labels
# ================================================================

# (If needed)
# !pip install -q earthengine-api geemap pandas matplotlib ipywidgets

import ee, geemap, pandas as pd, numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets

# --------------------------------------------
# 0) Initialize Earth Engine
# --------------------------------------------
# ee.Authenticate()  # uncomment if you didn't authenticate yet
ee.Initialize()
print("EE initialized.")


EE initialized.


In [3]:
# --------------------------------------------
# AREA OF INTEREST (AOI): Omo Forest Reserve
# --------------------------------------------
# --------------------------------------------
# 1) AREA OF INTEREST (AOI): Omo polygon you provided
# --------------------------------------------
Omo = ee.Geometry.Polygon([[
  [4.2361423319195035, 6.675272469761398],
  [4.2031833475445035, 6.604340319422325],
  [4.2691013162945035, 6.57705600965718],
  [4.3432590311382535, 6.598883577612744],
  [4.4009372537945035, 6.6425358246122395],
  [4.4366428202007535, 6.743466771144455],
  [4.5574924295757535, 6.808924273863115],
  [4.5739719217632535, 6.866192268040673],
  [4.5931979959820035, 7.005242878650024],
  [4.4805881327007535, 7.027050998402345],
  [4.3597385233257535, 7.09792030552165],
  [4.2471286600445035, 7.122449451452426],
  [4.1509982889507535, 7.057035491853953],
  [4.133191713488422, 6.975352088855542],
  [4.163404115832172, 6.794021458463974],
  [4.199109682238422, 6.742200066070055]
]])

AOI = Omo
AOI_SIMP = AOI.simplify(100)  # slight simplification helps reducers run faster
print("AOI set to custom Omo polygon.")





AOI set to custom Omo polygon.


In [4]:
# --------------------------------------------
# UTILS: Cloud masking & indices for Landsat-8 C2 L2
# --------------------------------------------
L8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')

L8_BANDS = ['SR_B2','SR_B3','SR_B4','SR_B5','SR_B6','SR_B7']  # Blue..SWIR2
OUT_BANDS = ['BLUE','GREEN','RED','NIR','SWIR1','SWIR2']

def scale_l2(img):
    # apply USGS scale/offset for SR bands
    sr = img.select(L8_BANDS).multiply(0.0000275).add(-0.2)
    qa = img.select('QA_PIXEL')
    return (sr.rename(OUT_BANDS)
            .copyProperties(img, img.propertyNames())
            .addBands(qa))

def mask_l2_clouds(img):
    qa = img.select('QA_PIXEL')
    # bit 3 = cloud shadow, bit 4 = cloud, bit 5 = snow, bit 7 = cirrus
    cloud_shadow = qa.bitwiseAnd(1<<3).eq(0)
    clouds       = qa.bitwiseAnd(1<<4).eq(0)
    snow         = qa.bitwiseAnd(1<<5).eq(0)
    cirrus       = qa.bitwiseAnd(1<<7).eq(0)
    mask = cloud_shadow.And(clouds).And(snow).And(cirrus)
    return img.updateMask(mask)

def add_indices(img):
    b = img
    ndvi = b.expression('(NIR-RED)/(NIR+RED)', {'NIR': b.select('NIR'),'RED': b.select('RED')}).rename('NDVI')
    ndwi = b.expression('(GREEN-NIR)/(GREEN+NIR)', {'GREEN': b.select('GREEN'),'NIR': b.select('NIR')}).rename('NDWI')
    ndbi = b.expression('(SWIR1-NIR)/(SWIR1+NIR)', {'SWIR1': b.select('SWIR1'),'NIR': b.select('NIR')}).rename('NDBI')
    evi  = b.expression('2.5*(NIR-RED)/(NIR+6*RED-7.5*BLUE+1)', {
        'NIR': b.select('NIR'), 'RED': b.select('RED'), 'BLUE': b.select('BLUE')
    }).rename('EVI')
    return img.addBands([ndvi, ndwi, ndbi, evi])

def annual_composite(year, region):
    start = ee.Date.fromYMD(year,1,1)
    end   = start.advance(1,'year')
    ic = (L8.filterDate(start, end)
              .filterBounds(region)
              .map(scale_l2)
              .map(mask_l2_clouds))
    # median composite
    comp = ic.median().select(OUT_BANDS).clip(region)
    return add_indices(comp)

print("Landsat utils ready.")


Landsat utils ready.


In [6]:
# ---- LANDSAT 8 L2 scaling + masking (fixed) ----
# Bands you want from L2 (surface reflectance)
L8_BANDS = ['SR_B2','SR_B3','SR_B4','SR_B5','SR_B6','SR_B7']  # Blue..SWIR2
OUT_BANDS = ['blue','green','red','nir','swir1','swir2']

def scale_l2(img):
    img = ee.Image(img)  # <-- IMPORTANT: cast to Image
    # Apply L2 scale factors: reflectance = DN * 0.0000275 + (-0.2)
    sr = img.select(L8_BANDS).multiply(0.0000275).add(-0.2).rename(OUT_BANDS)
    qa = img.select('QA_PIXEL')
    # keep QA as a band (useful for masking later/inspection)
    return sr.addBands(qa).copyProperties(img, img.propertyNames())

def mask_l2_clouds(img):
    img = ee.Image(img)  # <-- IMPORTANT: cast to Image
    qa = img.select('QA_PIXEL')
    # QA_PIXEL bit 3 = cloud shadow, bit 4 = cloud
    cloud_shadow = qa.bitwiseAnd(1 << 3).eq(0)
    cloud        = qa.bitwiseAnd(1 << 4).eq(0)
    mask = cloud.And(cloud_shadow)
    return img.updateMask(mask)

# Landsat 8 OLI/TIRS Collection 2, Tier 1, L2
L8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')


In [7]:
def annual_composite(year, region):
    start = ee.Date.fromYMD(year, 1, 1)
    end   = start.advance(1, 'year')
    ic = (L8.filterDate(start, end)
            .filterBounds(region)
            .map(scale_l2)
            .map(mask_l2_clouds))
    # Median composite of scaled bands, clip to AOI
    comp = ic.median().select(OUT_BANDS).clip(region)
    return comp


In [10]:
# --------------------------------------------
# BUILD 2020 & 2024 COMPOSITES
# --------------------------------------------
img2020 = annual_composite(2020, AOI)
img2024 = annual_composite(2024, AOI)
print("Composites built for 2020 & 2024.")



Composites built for 2020 & 2024.


In [11]:
# --------------------------------------------
# TRAINING DATA (proxy) FROM ESA Dynamic World
#    -> we map DW 9 classes to 6 target classes suitable for Landsat LULC
# --------------------------------------------
DW = ee.ImageCollection('GOOGLE/ESA/DYNAMICWORLD/V1')

# Map Dynamic World 0..8 -> our 6 classes
# 0 Water -> 0 Water
# 1 Trees, 6 Shrub & Scrub -> 1 Forest/Shrub
# 2 Grass, 4 Crops -> 2 Herbaceous/Cropland
# 3 Flooded vegetation -> 3 Wetlands
# 7 Bare ground -> 4 Bare
# 5 Built area -> 5 Built-up
DW2TARGET = ee.Dictionary({
    0: 0,                # Water
    1: 1, 6: 1,          # Forest/Shrub
    2: 2, 4: 2,          # Herbaceous/Cropland
    3: 3,                # Wetlands
    7: 4,                # Bare
    5: 5,                # Built-up
    8: 4                 # Snow & Ice -> treat as Bare in tropics (rare)
})

TARGET_CLASSES = {
    0: 'Water',
    1: 'Forest/Shrub',
    2: 'Herbaceous/Cropland',
    3: 'Wetlands',
    4: 'Bare',
    5: 'Built-up'
}
PALETTE = ['#4C9ED9','#1c6b2a','#B5D56A','#7A87C6','#C2B280','#C4281B']

def dw_year_mode(year, region):
    start = ee.Date.fromYMD(year,1,1)
    end = start.advance(1,'year')
    lab = (DW.filterDate(start,end)
             .filterBounds(region)
             .select('label')
             .reduce(ee.Reducer.mode())
             .select('label_mode')
             .rename('dw'))
    # remap to target classes
    remapped = lab.remap(
        # original class codes 0..8
        ee.List.sequence(0,8),
        ee.List([DW2TARGET.get(i) for i in ee.List.sequence(0,8).getInfo()])
    ).rename('label')
    return remapped.clip(region)

label2020 = dw_year_mode(2020, AOI)
label2024 = dw_year_mode(2024, AOI)

In [15]:
def as_label(img):
    """Ensure the label image has a single INT band called 'label'."""
    img = ee.Image(img)
    # If multiple bands slipped in, keep the first.
    first = ee.Image(img.select([0]))
    return first.rename('label').toInt()

label2020 = as_label(label2020)
label2024 = as_label(label2024)

In [16]:
# Make sure your TARGET_CLASSES is a dict like {0:'Water', 1:'Built', ...}
# If it exists already, we just sort the keys for consistency:
CLASS_VALUES = sorted(list(TARGET_CLASSES.keys()))

INPUT_BANDS = OUT_BANDS + ['NDVI','NDWI','NDBI','EVI']

def stratified_samples(image, label_img, region, n_per_class=400, seed=42):
    image = ee.Image(image).select(INPUT_BANDS)
    label_img = as_label(label_img).clip(region)

    # It helps to mask out places without labels so each feature has a valid 'label':
    image_for_sample = image.updateMask(label_img.mask())

    pts = (image_for_sample.addBands(label_img)
           .stratifiedSample(
               numPoints=n_per_class,
               classBand='label',
               region=region,
               scale=30,
               geometries=True,
               dropNulls=True,
               # Be explicit and stable about classes and per-class points
               classValues=CLASS_VALUES,
               classPoints=[n_per_class] * len(CLASS_VALUES)
           ))

    # Train/test split 70/30
    pts = pts.randomColumn('rand', seed)
    train = pts.filter(ee.Filter.lt('rand', 0.7))
    test  = pts.filter(ee.Filter.gte('rand', 0.7))
    return train, test

train2020, test2020 = stratified_samples(img2020, label2020, AOI_SIMP)
train2024, test2024 = stratified_samples(img2024, label2024, AOI_SIMP)

print('Sample sizes:',
      '2020 train', train2020.size().getInfo(), 'test', test2020.size().getInfo(),
      '2024 train', train2024.size().getInfo(), 'test', test2024.size().getInfo())


EEException: Dictionary.get, argument 'key': Invalid type.
Expected type: String.
Actual type: Integer.
Actual value: 0

In [ ]:
# --------------------------------------------
# ACCURACY (holdout) & CONFUSION MATRIX
# --------------------------------------------
def accuracy_report(classifier, test_fc):
    classified = test_fc.classify(classifier)
    cm = classified.errorMatrix('label', 'classification')
    return {
        'overall': cm.accuracy().getInfo(),
        'kappa': cm.kappa().getInfo(),
        'matrix': cm.getInfo()
    }

acc2020 = accuracy_report(clf2020, test2020)
acc2024 = accuracy_report(clf2024, test2024)

print("2020 accuracy:", acc2020['overall'], "kappa:", acc2020['kappa'])
print("2024 accuracy:", acc2024['overall'], "kappa:", acc2024['kappa'])

# --------------------------------------------
# MAPS: classification + side-by-side compare
# --------------------------------------------
legend_dict = {TARGET_CLASSES[i]: PALETTE[i] for i in TARGET_CLASSES.keys()}

def show_map(layer, title):
    m = geemap.Map()
    m.centerObject(AOI, 11)
    m.addLayer(layer, {'min':0,'max':5,'palette':PALETTE}, title)
    m.addLayer(ee.FeatureCollection([ee.Feature(AOI)]), {'color':'black'}, 'AOI')
    try: m.add_text(title, position='topcenter', font_size=16)
    except: pass
    m.add_legend(title="LULC (SVM)", legend_dict=legend_dict)
    m.addLayerControl()
    return m

# Single maps (uncomment to view individually)
# show_map(cls2020, "Omo Forest Reserve — LULC (SVM) 2020")
# show_map(cls2024, "Omo Forest Reserve — LULC (SVM) 2024")

# Side-by-side, linked maps
def linked_maps_2020_2024():
    lon, lat = AOI.centroid().coordinates().getInfo()
    center = (lat, lon)
    left  = geemap.Map(center=center, zoom=11)
    right = geemap.Map(center=center, zoom=11)
    left.addLayer(cls2020, {'min':0,'max':5,'palette':PALETTE}, 'LULC 2020 (SVM)')
    right.addLayer(cls2024, {'min':0,'max':5,'palette':PALETTE}, 'LULC 2024 (SVM)')
    outline = ee.FeatureCollection([ee.Feature(AOI)])
    left.addLayer(outline, {'color':'black'}, 'AOI'); right.addLayer(outline, {'color':'black'}, 'AOI')
    try: left.add_text("2020", position="topcenter"); right.add_text("2024", position="topcenter")
    except: pass
    left.add_legend(title="Classes", legend_dict=legend_dict)
    right.add_legend(title="Classes", legend_dict=legend_dict)
    geemap.link(left, right)
    return widgets.HBox([left, right])

linked_maps_2020_2024()

### Area by class (hectares) + side-by-side bars

In [ ]:
# --------------------------------------------
# AREA BY CLASS (ha) with grouped reducer (fast)
# --------------------------------------------
def area_ha_by_class(class_img, region, scale=30, tilescale=2):
    area = ee.Image.pixelArea().rename('area')
    pair = area.addBands(class_img.rename('class')).clip(region)
    red = pair.reduceRegion(
        reducer=ee.Reducer.sum().group(groupField=1, groupName='class'),
        geometry=region, scale=scale, bestEffort=True, maxPixels=2e13, tileScale=tilescale
    )
    groups = ee.Dictionary(red).get('groups')
    groups = ee.List(groups).getInfo() if groups is not None else []
    data = {int(g['class']): g['sum']/10000.0 for g in groups}   # m² -> ha
    # ensure all classes present
    vals = [data.get(i,0.0) for i in TARGET_CLASSES.keys()]
    return pd.Series(vals, index=[TARGET_CLASSES[i] for i in TARGET_CLASSES.keys()], name='Hectares')

ha2020 = area_ha_by_class(cls2020, AOI_SIMP, scale=30, tilescale=2)
ha2024 = area_ha_by_class(cls2024, AOI_SIMP, scale=30, tilescale=2)

display(ha2020.to_frame('2020').style.format('{:,.0f}'))
display(ha2024.to_frame('2024').style.format('{:,.0f}'))

# Side-by-side bar chart (shared y-axis)
def side_by_side_bars(s2020, s2024, title="Omo Forest Reserve — LULC Area (ha)"):
    cats = list(s2020.index)
    ymax = max(s2020.max(), s2024.max())*1.1
    fig, axes = plt.subplots(1,2, figsize=(14,5), sharey=True)
    axes[0].bar(cats, s2020.values); axes[0].set_title('2020'); axes[0].set_ylabel('Hectares')
    axes[0].set_xticklabels(cats, rotation=30, ha='right'); axes[0].set_ylim(0,ymax)
    axes[1].bar(cats, s2024.values); axes[1].set_title('2024')
    axes[1].set_xticklabels(cats, rotation=30, ha='right'); axes[1].set_ylim(0,ymax)
    fig.suptitle(title); plt.tight_layout(); plt.show()

side_by_side_bars(ha2020, ha2024)

### Export

In [ ]:
# --------------------------------------------
# OPTIONAL EXPORTS (Google Drive)
# --------------------------------------------

# Export classified rasters (GeoTIFF, 10/30 m) – adjust scale as needed
# geemap.ee_export_image(cls2020.clip(AOI), filename='Omo_LULC_SVM_2020.tif', scale=30, region=AOI, file_per_band=False)
# geemap.ee_export_image(cls2024.clip(AOI), filename='Omo_LULC_SVM_2024.tif', scale=30, region=AOI, file_per_band=False)

# Export area tables
df_compare = pd.concat([ha2020.rename('2020'), ha2024.rename('2024')], axis=1)
df_compare.to_csv('Omo_LULC_area_ha_2020_vs_2024.csv', float_format='%.0f')
print("Saved: Omo_LULC_area_ha_2020_vs_2024.csv")

# Export accuracy reports to JSON/text
import json, pathlib
pathlib.Path('outputs').mkdir(exist_ok=True)
with open('outputs/accuracy_2020.json','w') as f: json.dump(acc2020, f, indent=2)
with open('outputs/accuracy_2024.json','w') as f: json.dump(acc2024, f, indent=2)
print("Saved: outputs/accuracy_*.json")
